# 少量示例的提示词模板的使用

FewShotPromptTemplate: 与PromptTemplate一起使用

FewShotChatMessagePromptTemplate：与ChatPromptTemplate一起使用

Example selectors(示例选择器):

## 1、FewShotPromptTemplate的使用


举例1：未提供示例的情况

In [1]:

import os
import dotenv
from langchain_core.prompts import FewShotPromptTemplate
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"),
                        temperature=0.7)

res = chat_model.invoke("2 🦜 9是多少?")
print(res.content)

您的表达可能需要一些澄清。如果您是想要进行数学运算，那么2加上9等于11。如果您是指其他含义，请提供更多信息，我将很乐意帮助您。


举例2：使用FewShotPromptTemplate



In [24]:
from langchain_core.prompts import PromptTemplate

# 创建PromptTemplate的实例
example_prompt = PromptTemplate.from_template(
    template="input:{input}\noutput:{output}",
)

# 提供一些示例
examples = [
    {"input": "长沙天气怎么样？", "output": "长沙市"},
    {"input": "南京下雨吗？", "output": "南京市"},
    {"input": "武汉热吗？", "output": "武汉市"}
]

# 创建FewShotPromptTemplate的实例
few_shot_template = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,
    suffix="input:{input}\noutput:", # 声明在示例后面的提示词模板
    input_variables=["input"],
)

few_shot_template.invoke({"input":"杭州会下雨吗？"})

StringPromptValue(text='input:长沙天气怎么样？\noutput:长沙市\n\ninput:南京下雨吗？\noutput:南京市\n\ninput:武汉热吗？\noutput:武汉市\n\ninput:杭州会下雨吗？\noutput:')

调用大模型以后：

In [27]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"), temperature=0.4)

chat_model.invoke(few_shot_template.invoke({"input":"杭州会下雨吗？"}))


AIMessage(content='杭州市', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 68, 'total_tokens': 69, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': '', 'id': '019bf409bae898789574b7e88db76545', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bf409-c784-7fd1-90a7-9919f0684162-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 68, 'output_tokens': 1, 'total_tokens': 69, 'input_token_details': {}, 'output_token_details': {}})

举例3：


In [3]:
from langchain_core.prompts import PromptTemplate

# 提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数学专家,算式： {input} 值： {output} 使用： {description} "

# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)

# 2、提供示例
examples = [
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]

# 3、创建一个FewShotPromptTemplate对象
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_sample,
    suffix="你是一个数学专家,算式: {input}  值: {output}",
    input_variables=["input", "output"]
)
# print(prompt.invoke({"input":"2*5", "output":"10"}))

# 4、初始化大模型，然后调用
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

result = chat_model.invoke(prompt.invoke({"input":"2*5", "output":""}))
print(result.content)  # 使用: 乘法运算

你是一个数学专家,算式: 2*5 值: 10 使用： 乘法运算 

解析：这个算式是2乘以5，根据乘法运算的规则，2乘以5的结果是10。在这个例子中，2是乘数，5也是乘数，它们相乘的结果就是10。


# 2、FewShotChatMessagePromptTemplate的使用

举例1：实例化